# Churn Modelling

In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

## Preprocessing 

In [56]:
#Load the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [57]:
# Drop unwanted and irrelevant fields
dataset = dataset.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1) # axis=1 specifies we want to drop columns, o is for rows and is default.
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Encoding the categorical data

In [58]:
# Using LabelEncoder to encode binary gender data
le = LabelEncoder()
gender_encoding = le.fit(dataset['Gender'])
dataset['Gender'] = gender_encoding.transform(dataset['Gender'])
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [64]:
# Using OneHotEncoder to encode the 'Geography' categorical data
ohe = OneHotEncoder(sparse_output=False) # sparse_output=False, to not create a sparse matrix of the encoded data
geo_encoding = ohe.fit(dataset[['Geography']])
encoded_geographies = geo_encoding.transform(dataset[['Geography']])
encoded_geographies_to_dataframe = pd.DataFrame(encoded_geographies, columns=ohe.get_feature_names_out(['Geography']))
dataset = dataset.drop(['Geography'],axis=1)
## Combine one hot encoder columns with the original data
dataset = pd.concat([dataset,encoded_geographies_to_dataframe],axis=1)
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [65]:
# Divide the dataset into dependent and independent features
X = dataset.drop(['Exited'],axis=1)
y = dataset['Exited']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scalling the features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [66]:
# Saving the data into a serialized pickle format
with open('dataset.pkl','wb') as file:
    pickle.dump(dataset,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(sc,file)

with open('geo_encoding.pkl','wb') as file:
    pickle.dump(geo_encoding,file)

with open('gender_encoding.pkl','wb') as file:
    pickle.dump(gender_encoding,file)



## ANN Implementation

### Steps:
#### 1. Build ANN model
    - Input layer -> 12 neurons
    - Hidden layer1 -> 64 neurons -> relu activation
    - Hidden layer2 -> 32 neurons -> relu activation
    - Output layer -> 1 neuron -> sigmoid activation

#### 2. Initiate optimizer and loss functions
    - Optimizer -> Adam -> learning rate=0.01
    - Loss function -> Binary cross entropy 
#### 3. Compile the model
    - With optimizer, loss, and accuracy
#### 4. Setup callbacks
    - Early stopping -> monitor=val_loss, patience=10, restore best weights=true
    - Tensorboard
#### 5. Fit model
    - With training, validation data
    - Epochs -> 100
    - Callbacks -> Early stopping, Tensorboard
#### 6. Save the model
    - In h5 format
#### 7. Preview using tensorboard
    - logs/fit

In [41]:
# Importing libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime as dt

In [42]:
# Building model
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# Initializing optimizer and loss function
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
opt = Adam(learning_rate=0.01)
loss = BinaryCrossentropy()

In [45]:
# Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [46]:
# Setup callbacks 
logs_dir = "logs/fit/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [47]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=50,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7966 - loss: 0.4440 - val_accuracy: 0.8395 - val_loss: 0.3680
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8465 - loss: 0.3624 - val_accuracy: 0.8600 - val_loss: 0.3454
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8567 - loss: 0.3440 - val_accuracy: 0.8540 - val_loss: 0.3510
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8641 - loss: 0.3414 - val_accuracy: 0.8585 - val_loss: 0.3438
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8647 - loss: 0.3332 - val_accuracy: 0.8595 - val_loss: 0.3469
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8678 - loss: 0.3341 - val_accuracy: 0.8565 - val_loss: 0.3406
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8557 - loss: 0.3453 - val_accuracy: 0.8540 - val_loss: 0.3467
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8616 - loss: 0.3405 - val_accuracy: 0.

In [48]:
# Save the model
model.save("churn_model.h5")
model.save("churn_model.keras")

In [ ]:
# Load Tensorboard extension
%load_ext tensorboard
%tensorboard --logdir logs/fit